In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/book-text/texts.csv


In [2]:
dt = pd.read_csv("/kaggle/input/book-text/texts.csv")

In [3]:
dt

,url,content,name
0,https://gutenberg.org/cache/epub/6130/pg6130.txt,﻿The Project Gutenberg eBook of The Iliad\r\n ...,Iliad
1,https://ia601606.us.archive.org/25/items/pdfy-...,The Corpus Hermeticum\ntranslated by G.R.S. M...,Corpus Hermeticum
2,https://openbible.com/textfiles/kjv.txt,ï»¿KJV\nKing James Bible: Pure Cambridge Editi...,Bible
3,https://www.gutenberg.org/files/2388/2388-h/23...,\n\n\n\n\r\nThe Project Gutenberg E-text of Th...,Bhagavad Geeta
4,https://www.avesta.org/kanga/ka_english_kanga_...,"KHORDEH A VEST Ā\nComprising\nAshem, Yatha, th...",Khordeh Avesta
5,https://archive.org/stream/RigVedaSanhitaByHHW...,"\n\n\n\nFull text of ""Rig Veda Sanhita By H H ...",Rigveda
6,https://www.gutenberg.org/files/2017/2017-h/20...,"\n\n\n\n\n The Dhammapada, by an Unknown ...",Dhammapada


In [4]:
import re

def preprocess(text):
    text = re.sub(r'[^a-zA-Z]',' ',text)
    text = ' '.join(text.split())
    return text

In [5]:
dt['text'] = dt['content'].apply(preprocess)

In [6]:
dt

,url,content,name,text
0,https://gutenberg.org/cache/epub/6130/pg6130.txt,﻿The Project Gutenberg eBook of The Iliad\r\n ...,Iliad,The Project Gutenberg eBook of The Iliad This ...
1,https://ia601606.us.archive.org/25/items/pdfy-...,The Corpus Hermeticum\ntranslated by G.R.S. M...,Corpus Hermeticum,The Corpus Hermeticum translated by G R S Mead...
2,https://openbible.com/textfiles/kjv.txt,ï»¿KJV\nKing James Bible: Pure Cambridge Editi...,Bible,KJV King James Bible Pure Cambridge Edition Te...
3,https://www.gutenberg.org/files/2388/2388-h/23...,\n\n\n\n\r\nThe Project Gutenberg E-text of Th...,Bhagavad Geeta,The Project Gutenberg E text of The Bhagavad G...
4,https://www.avesta.org/kanga/ka_english_kanga_...,"KHORDEH A VEST Ā\nComprising\nAshem, Yatha, th...",Khordeh Avesta,KHORDEH A VEST Comprising Ashem Yatha the five...
5,https://archive.org/stream/RigVedaSanhitaByHHW...,"\n\n\n\nFull text of ""Rig Veda Sanhita By H H ...",Rigveda,Full text of Rig Veda Sanhita By H H Wilson Vo...
6,https://www.gutenberg.org/files/2017/2017-h/20...,"\n\n\n\n\n The Dhammapada, by an Unknown ...",Dhammapada,The Dhammapada by an Unknown Author The Projec...


In [ ]:
from transformers import pipeline

summarizer = pipeline('summarization', model='facebook/bart-large-cnn')

In [ ]:
summarizer(text[1000:2000],max_length = 50,min_length=20,do_sample=False)

In [ ]:
#dt['themes'] = dt['text'].apply(lambda x: summarizer(x, max_length=150, min_length=50, do_sample=False))

In [45]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 8.0 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

dt['embeddings'] = dt['text'].apply(lambda x: model.encode(x))

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(dt['embeddings'].tolist())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.heatmap(similarity_matrix, annot=True, cmap='coolwarm')
plt.title('Thematic Similarity')
plt.show()

In [ ]:
dt['name']

In [ ]:
from wordcloud import WordCloud

wordcloud = WordCloud().generate(' '.join(dt['text']))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
dt

In [7]:
bible=dt.loc[2,'content']

In [11]:
import re
from collections import defaultdict
pattern = r"([a-zA-Z]+) (\d+):(\d+)\t(.+)"

# Dictionary to hold chapters and verses
bible_structure = defaultdict(lambda: defaultdict(list))

# Process the text
for match in re.finditer(pattern, bible):
    book, chapter, verse, verse_text = match.groups()
    bible_structure[book][int(chapter)].append((int(verse), verse_text.strip()))


In [ ]:
# Print the structured output
# for book, chapters in bible_structure.items():
#     print(f"Book: {book}")
#     for chapter, verses in sorted(chapters.items()):
#         print(f"  Chapter {chapter}:")
#         for verse_num, verse_text in sorted(verses):
#             print(f"    {verse_num}: {verse_text}")

In [ ]:
#bible_structure['Genesis'][2]

In [ ]:
#!pip install langchain
!pip install langchain_openai

In [ ]:
bible[:1000]

In [8]:
from collections import defaultdict
import re

pattern = r"([a-zA-Z]+) (\d+):(\d+)\t(.+)"
bible_dict = defaultdict(lambda : defaultdict(str))

for match in re.finditer(pattern,bible):
    book, ch, ver, text = match.groups()
    
    bible_dict[book][int(ch)]+=f" {text.strip()}"

In [12]:
#bible_dict['Mark']

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Initialize the model and tokenizer
model_name = "t5-base"  # You can also use "t5-large" or "t5-3b" for larger models
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

def summarize_text(text, max_length=150):
    # Prepare input by adding the prefix "summarize: "
    input_text = f"summarize: {text}"
    
    # Tokenize the input text
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding="longest")
    
    # Generate the summary
    summary_ids = model.generate(inputs["input_ids"], max_length=max_length, num_beams=4, early_stopping=True)
    
    # Decode and return the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Example usage: Summarizing a chunk of the Bible (Genesis chapter)
genesis_chunk = bible_dict['Mark'][1]
# Summarize this chunk
genesis_summary = summarize_text(genesis_chunk)
print("Genesis Summary:", genesis_summary)


## Abstractive Summary

In [11]:
import torch
from transformers import pipeline

hf_name = 'pszemraj/led-large-book-summary'

summarizer = pipeline(
    "summarization",
    hf_name,
    device=0 if torch.cuda.is_available() else -1,
)

config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.84G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [12]:
wall_of_text = bible_dict['Mark'][1]

result = summarizer(
    wall_of_text,
    min_length=16,
    max_length=256,
    no_repeat_ngram_size=3,
    encoder_no_repeat_ngram_size=3,
    repetition_penalty=3.5,
    num_beams=4,
    early_stopping=True,
)

Input ids are automatically padded from 1195 to 2048 to be a multiple of `config.attention_window`: 1024


In [13]:
result[0]['summary_text']

'The first chapter begins with a prologue that traces the story of the Gospel of Mark, the "Book of Matthew. The prologue is a message from God to Matthew, Jesus\'s messenger, who says something like this: Dear son, I\'m going to show you the way to the kingdom. Here\'s the deal: John the Baptist preacher did a lot of preaching during the wilderness in the Bible. He preached about repentance and the holiness of the body through baptismal baptism for people who had committed sins. There was lots of mingling of bodies and lots of eating of honey and locust until Jesus showed up at the Jordan River and baptized John\'s brother, John, full of camel\'s hair. Jesus then baptized John, but he got baptized with water but Jesus baptized him with the holy spirit. This is where the story gets a little weird. Jesus comes from Galilee and baptizes all of the townsfolk in the region as they come to confess their sins, but none of them are able to wash their sins away. Then he goes out into the world

In [ ]:
def summarizer(text):
    result = summarizer(
        text,
        min_length=16,
        max_length=256,
        no_repeat_ngram_size=3,
        encoder_no_repeat_ngram_size=3,
        repetition_penalty=3.5,
        num_beams=4,
        early_stopping=True,
    )
    return result

In [16]:
summ = []
for ch in bible_dict['Mark']:
    text = ""
    t = (bible_dict['Mark'][ch])
    text+=f" {t.strip()}"
    res = summarizer(text)[0]['summary_text']
    summ.append(res)

Your max_length is set to 1024, but your input_length is only 923. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=461)
Input ids are automatically padded from 923 to 1024 to be a multiple of `config.attention_window`: 1024
Your max_length is set to 1024, but your input_length is only 880. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=440)
Input ids are automatically padded from 880 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 1167 to 2048 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 1225 to 2048 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 1671 to 2048 to be a multiple of `conf

In [13]:
#summ

## Semantic Search

In [41]:
corpus = []
for book, chapters in bible_structure.items():
    for chapter, verses in sorted(chapters.items()):
        for verse_num, verse_text in sorted(verses):
            corpus.append(verse_text)

In [43]:
corpus[:10]

['In the beginning God created the heaven and the earth.',
 'And the earth was without form, and void; and darkness [was] upon the face of the deep. And the Spirit of God moved upon the face of the waters.',
 'And God said, Let there be light: and there was light.',
 'And God saw the light, that [it was] good: and God divided the light from the darkness.',
 'And God called the light Day, and the darkness he called Night. And the evening and the morning were the first day.',
 'And God said, Let there be a firmament in the midst of the waters, and let it divide the waters from the waters.',
 'And God made the firmament, and divided the waters which [were] under the firmament from the waters which [were] above the firmament: and it was so.',
 'And God called the firmament Heaven. And the evening and the morning were the second day.',
 'And God said, Let the waters under the heaven be gathered together unto one place, and let the dry [land] appear: and it was so.',
 'And God called the dry

In [46]:
import torch

from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer("all-MiniLM-L6-v2")

corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# Query sentences:
queries = [
    "when did god create heavens and earth",
    "who is jesus",
    "where is the ark of covenant",
]

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(5, len(corpus))
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    similarity_scores = embedder.similarity(query_embedding, corpus_embeddings)[0]
    scores, indices = torch.topk(similarity_scores, k=top_k)

    print("\nQuery:", query)
    print("Top 5 most similar sentences in corpus:")

    for score, idx in zip(scores, indices):
        print(corpus[idx], f"(Score: {score:.4f})")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...


Query: when did god create heavens and earth
Top 5 most similar sentences in corpus:
In the beginning God created the heaven and the earth. (Score: 0.7475)
These [are] the generations of the heavens and of the earth when they were created, in the day that the LORD God made the earth and the heavens, (Score: 0.6831)
For thus saith the LORD that created the heavens; God himself that formed the earth and made it; he hath established it, he created it not in vain, he formed it to be inhabited: I [am] the LORD; and [there is] none else. (Score: 0.6817)
The LORD by wisdom hath founded the earth; by understanding hath he established the heavens. (Score: 0.6193)
He hath made the earth by his power, he hath established the world by his wisdom, and hath stretched out the heavens by his discretion. (Score: 0.5934)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Query: who is jesus
Top 5 most similar sentences in corpus:
Who is he that overcometh the world, but he that believeth that Jesus is the Son of God? (Score: 0.7190)
Among whom are ye also the called of Jesus Christ: (Score: 0.6325)
Whosoever believeth that Jesus is the Christ is born of God: and every one that loveth him that begat loveth him also that is begotten of him. (Score: 0.6054)
Whosoever shall confess that Jesus is the Son of God, God dwelleth in him, and he in God. (Score: 0.5796)
Who is a liar but he that denieth that Jesus is the Christ? He is antichrist, that denieth the Father and the Son. (Score: 0.5791)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Query: where is the ark of covenant
Top 5 most similar sentences in corpus:
And I have set there a place for the ark, wherein [is] the covenant of the LORD, which he made with our fathers, when he brought them out of the land of Egypt. (Score: 0.7526)
And in it have I put the ark, wherein [is] the covenant of the LORD, that he made with the children of Israel. (Score: 0.7184)
Behold, the ark of the covenant of the Lord of all the earth passeth over before you into Jordan. (Score: 0.6746)
And the children of Israel inquired of the LORD, (for the ark of the covenant of God [was] there in those days, (Score: 0.6286)
Now set your heart and your soul to seek the LORD your God; arise therefore, and build ye the sanctuary of the LORD God, to bring the ark of the covenant of the LORD, and the holy vessels of God, into the house that is to be built to the name of the LORD. (Score: 0.6246)
